In [2]:
import requests
import pandas as pd
from lookups import DataSourcesAPI, DataSourcesIncome
from io import StringIO

In [5]:
response = requests.get(DataSourcesIncome.PER_CAPITA_AUSTIN_INCOME.value)
if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text))
else:
    print(f'Failed to download data. Status code: {response.status_code}')

In [6]:
df.head()

,DATE,PCPI48015
0,1969-01-01,2852
1,1970-01-01,3259
2,1971-01-01,3701
3,1972-01-01,4239
4,1973-01-01,4421


In [7]:
df.iloc[:,[1]].head()

,PCPI48015
0,2852
1,3259
2,3701
3,4239
4,4421


In [8]:
df.iloc[[0]]

,DATE,PCPI48015
0,1969-01-01,2852


In [22]:
DataSourcesIncome.PER_CAPITA_AUSTIN_INCOME.name.split('_')[2].lower()

'austin'

In [64]:
def readIncomeData():
    df = pd.DataFrame()
    for item in DataSourcesIncome:
        response = requests.get(item.value)
        if response.status_code == 200:
            if item.name == DataSourcesIncome.PER_CAPITA_SONOMA_INCOME.name:
                df = pd.concat([df,pd.read_csv(StringIO(response.text))])
                df.columns = ['date','Sonoma']
            else:
                df[item.name.split('_')[2].title()] = pd.read_csv(StringIO(response.text)).iloc[:,[1]]
    return pd.melt(df, id_vars=['date'], var_name='region', value_name='personal_income')

In [65]:
income_df = readIncomeData()

In [66]:
income_df.head()

,date,region,personal_income
0,1969-01-01,Sonoma,4298
1,1970-01-01,Sonoma,4646
2,1971-01-01,Sonoma,5006
3,1972-01-01,Sonoma,5380
4,1973-01-01,Sonoma,5760


In [10]:
def readIncomeData():
    income_dict = dict()
    for item in DataSourcesIncome:
        response = requests.get(item.value)
        if response.status_code == 200:
            df = pd.read_csv(StringIO(response.text))
            df.columns = ['date',item.name.split('_')[2].title()]
            income_dict[item.name.split('_')[2].title()] = df
    return income_dict

In [11]:
dict_ = readIncomeData()

In [14]:
dict_.keys()

dict_keys(['Sonoma', 'Austin', 'Norfolk', 'Bloomington', 'Dallas'])